In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from obspy import read
from scipy import signal
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta
import json
from sklearn.metrics import accuracy_score, mean_absolute_error, ConfusionMatrixDisplay

# Set device to CPU
device = torch.device("cpu")

# Directory paths
lunar_catalog_path = '../../data/lunar_data/training/catalogs/apollo12_catalog_GradeA_final.csv'
lunar_data_directory = '../../data/lunar_data/training/data/S12_GradeA/'
martian_data_directory = '../../data/marsquake_data/training/data/'
lunar_data_images_dir = '../../model/model_output/lunar_preprocessed_images/'
martian_data_images_dir = '../../model/model_output/martian_preprocessed_images/'

In [2]:
# Utility Functions
def convert_rel_to_abs_time(start_time, time_rel):
    return (start_time + timedelta(seconds=float(time_rel))).strftime('%Y-%m-%dT%H:%M:%S.%f')

def apply_bandpass_filter(trace, sampling_rate, freqmin=0.5, freqmax=3.0):
    sos = signal.butter(4, [freqmin, freqmax], btype='bandpass', fs=sampling_rate, output='sos')
    return signal.sosfilt(sos, trace)

def load_existing_images(image_dir):
    image_files = [os.path.join(root, file) for root, _, files in os.walk(image_dir) for file in files if file.endswith('.png')]
    return image_files

In [3]:
# Preprocessing Functions
def preprocess_lunar_data(lunar_data_directory):
    lunar_images = load_existing_images(lunar_data_directory)
    print(f"{len(lunar_images)} lunar images loaded.")
    return lunar_images

def preprocess_martian_data(data_dir, save_dir, combine_images=True):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    martian_images, martian_arrival_times = [], []
    mseed_files = [f for f in os.listdir(data_dir) if f.endswith('.mseed')]

    if len(mseed_files) == 0:
        print("No .mseed files found in the directory.")
        return martian_images, martian_arrival_times

    for filename in mseed_files:
        file_path = os.path.join(data_dir, filename)
        csv_file_path = file_path.replace('.mseed', '.csv')

        if not os.path.exists(csv_file_path):
            print(f"CSV file not found for {filename}: {csv_file_path}")
            continue

        try:
            csv_data = pd.read_csv(csv_file_path)
            arrival_time_rel = csv_data['rel_time(sec)'].iloc[0]
            image_path = plot_and_save_trace_spectrogram(file_path, arrival_time_rel, save_dir, filename, combine_images)
            martian_images.append(image_path)
            martian_arrival_times.append(arrival_time_rel)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    return martian_images, martian_arrival_times

In [4]:
# CNN Model Definition
class SpectrogramCNN(nn.Module):
    def __init__(self):
        super(SpectrogramCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 112 * 112, 128) #edit to None if it doesn't work
        self.fc_event = nn.Linear(128, 3)
        self.fc_time = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), 128)
        x = torch.relu(self.fc1(x))
        return self.fc_event(x), self.fc_time(x)

In [5]:
# Data Preparation
def prepare_data_for_training(image_files, labels, time_labels, batch_size=32):
    if not image_files:
        return None  # Early exit if no images are provided
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image_tensors = [transform(Image.open(img)) for img in image_files if os.path.exists(img) and img.endswith('.png')]
    if image_tensors:
        X_tensor = torch.stack(image_tensors)
        y_event_tensor = torch.tensor(labels, dtype=torch.long)
        y_time_tensor = torch.tensor(time_labels, dtype=torch.float32)
        dataset = TensorDataset(X_tensor, y_event_tensor, y_time_tensor)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return None

# Training Functions
def train_model(model, train_loader, criterion_event, criterion_time, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, event_labels, time_labels in train_loader:
            optimizer.zero_grad()
            event_output, time_output = model(inputs)
            loss_event = criterion_event(event_output, event_labels)
            loss_time = criterion_time(time_output.squeeze(), time_labels)
            loss = loss_event + loss_time
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

def self_train_on_martian_data(model, martian_data_loader, optimizer, criterion_event, criterion_time, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in martian_data_loader:
            images = batch[0]  # Unpack the images from the batch tuple
            optimizer.zero_grad()

            # Forward pass through the model
            event_output, time_output = model(images)

            # Generate pseudo-labels (predicted event labels)
            _, pseudo_labels = torch.max(event_output, 1)

            # For now, we do not have ground truth for Martian data, so we only optimize on pseudo-labels
            loss_event = criterion_event(event_output, pseudo_labels)
            loss_time = criterion_time(time_output.squeeze(), torch.zeros_like(time_output.squeeze()))  # Use zeros as placeholder

            # Calculate total loss and perform backpropagation
            total_loss = loss_event + loss_time
            total_loss.backward()
            optimizer.step()

            running_loss += total_loss.item()

        print(f"Self-training Epoch {epoch+1}, Loss: {running_loss/len(martian_data_loader)}")

# Model Evaluation
def evaluate_model(model, data_loader):
    model.eval()
    event_preds, time_preds, event_true, time_true = [], [], [], []
    with torch.no_grad():
        for images, event_labels, time_labels in data_loader:
            event_output, time_output = model(images)
            _, event_pred_classes = torch.max(event_output, 1)
            event_preds.extend(event_pred_classes.cpu().numpy())
            time_preds.extend(time_output.cpu().numpy())
            event_true.extend(event_labels.cpu().numpy())
            time_true.extend(time_labels.cpu().numpy())
    event_accuracy = accuracy_score(event_true, event_preds)
    time_mae = mean_absolute_error(time_true, time_preds)
    print(f"Validation Event Accuracy: {event_accuracy:.4f}")
    print(f"Validation Time MAE: {time_mae:.4f}")
    ConfusionMatrixDisplay.from_predictions(event_true, event_preds)
    plt.show()

In [6]:
# Model Save Function
def save_model_artifacts(model, model_name='seismic_cnn_model'):
    model_architecture = {
        'conv_layers': [{'in_channels': model.conv1.in_channels, 'out_channels': model.conv1.out_channels, 'kernel_size': model.conv1.kernel_size},
                        {'in_channels': model.conv2.in_channels, 'out_channels': model.conv2.out_channels, 'kernel_size': model.conv2.kernel_size}],
        'fc_layers': [{'in_features': model.fc_event.in_features, 'out_features': model.fc_event.out_features},
                      {'in_features': model.fc_time.in_features, 'out_features': model.fc_time.out_features}]
    }
    with open(f'{model_name}_architecture.json', 'w') as f:
        json.dump(model_architecture, f, indent=4)
    torch.save(model.state_dict(), f'{model_name}_weights.pth')
    torch.save(model, f'{model_name}_full.pth')
    print(f"Model saved to {model_name}_full.pth")

In [7]:
def flatten_image_list(image_list):
    """
    Ensure image list is flat in case there are nested lists of image paths.
    """
    if isinstance(image_list, (list, tuple)) and any(isinstance(i, (list, tuple)) for i in image_list):
        return [item for sublist in image_list for item in sublist]
    return image_list

In [8]:
def prepare_unlabeled_data_loader(image_files, batch_size=32):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image_tensors = [transform(Image.open(img)) for img in image_files if os.path.exists(img) and img.endswith('.png')]
    if image_tensors:
        X_tensor = torch.stack(image_tensors)
        dataset = TensorDataset(X_tensor)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return None

In [9]:
def main():
    # Define the paths for preprocessed images
    lunar_data_images_dir = '../../model/model_output/lunar_preprocessed_images/'
    martian_data_images_dir = '../../model/model_output/martian_preprocessed_images/'
    save_dir = martian_data_images_dir  # Directory where Martian images are saved

    # 1. Load pre-generated lunar images
    print(f"Loading pre-generated lunar data from: {lunar_data_images_dir}")
    lunar_images = preprocess_lunar_data(lunar_data_images_dir)

    # Check if any lunar images were loaded
    if not lunar_images:
        print("Error: No lunar images found. Exiting.")
        return  # Exit early if no images found

    lunar_labels = [0] * len(lunar_images)  # Placeholder labels for lunar data
    lunar_times = [0] * len(lunar_images)   # Placeholder arrival times for lunar data

    # Prepare DataLoader for lunar data
    print("Preparing DataLoader for lunar data...")
    train_loader = prepare_data_for_training(lunar_images, lunar_labels, lunar_times)

    if train_loader is None:
        print("Error: No valid data for training. DataLoader creation failed.")
        return  # Exit early if DataLoader creation failed

    # 2. Initialize and train the model on lunar data
    print("Initializing SpectrogramCNN model...")
    model = SpectrogramCNN()
    criterion_event = nn.CrossEntropyLoss()
    criterion_time = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print("Training model on lunar data...")
    train_model(model, train_loader, criterion_event, criterion_time, optimizer)

    # 3. Load pre-generated Martian images
    print(f"Loading pre-generated Martian data from: {martian_data_images_dir}")
    martian_images = load_existing_images(martian_data_images_dir)  # Load images directly

    # Check if any Martian images were loaded
    if not martian_images:
        print("Error: No Martian images found. Exiting.")
        return  # Exit early if no images found

    # Prepare DataLoader for Martian data
    print("Preparing DataLoader for Martian data...")
    martian_data_loader = prepare_unlabeled_data_loader(martian_images)

    if martian_data_loader is None:
        print("Error: No valid data for self-training. DataLoader creation failed.")
        return  # Exit early if DataLoader creation failed

    # 4. Self-train the model on Martian data
    print("Self-training model on Martian data...")
    self_train_on_martian_data(model, martian_data_loader, optimizer, criterion_event, criterion_time)

    # 5. Save the fine-tuned model
    print("Saving the fine-tuned model...")
    save_model_artifacts(model, model_name='../../model/fine_tuned_martian_seismic_cnn')


In [10]:
if __name__ == "__main__":
    main()

Loading pre-generated lunar data from: ../../model/model_output/lunar_preprocessed_images/
76 lunar images loaded.
Preparing DataLoader for lunar data...
Initializing SpectrogramCNN model...
Training model on lunar data...
Epoch 1, Loss: 1600.0661470890045
Epoch 2, Loss: 449.5803413391113
Epoch 3, Loss: 195.09067153930664
Epoch 4, Loss: 3.13076926022768
Epoch 5, Loss: 0.5083693067232767
Epoch 6, Loss: 0.670348584651947
Epoch 7, Loss: 0.8254997730255127
Epoch 8, Loss: 0.9365084966023763
Epoch 9, Loss: 0.9263431032498678
Epoch 10, Loss: 0.9064249992370605
Loading pre-generated Martian data from: ../../model/model_output/martian_preprocessed_images/
Preparing DataLoader for Martian data...
Self-training model on Martian data...
Self-training Epoch 1, Loss: 0.8976970314979553
Self-training Epoch 2, Loss: 0.889222264289856
Self-training Epoch 3, Loss: 0.8802964687347412
Self-training Epoch 4, Loss: 0.8709366917610168
Self-training Epoch 5, Loss: 0.8611681461334229
Self-training Epoch 6, Los

Code Analysis and Function Explanations
Imports and Setup

```python

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from obspy import read
from scipy import signal
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta
import json
from sklearn.metrics import accuracy_score, mean_absolute_error, ConfusionMatrixDisplay

# Set device to CPU
device = torch.device("cpu")

# Directory paths
lunar_catalog_path = '../../data/lunar_data/training/catalogs/apollo12_catalog_GradeA_final.csv'
lunar_data_directory = '../../data/lunar_data/training/data/S12_GradeA/'
martian_data_directory = '../../data/marsquake_data/training/data/'
lunar_data_images_dir = '../../model/model_output/lunar_preprocessed_images/'
martian_data_images_dir = '../../model/model_output/martian_preprocessed_images/'
```
Explanation:

    The code imports necessary libraries for data manipulation (numpy, pandas), signal processing (scipy, obspy), image processing (PIL, torchvision), machine learning (torch, sklearn), and visualization (matplotlib).
    Sets the computation device to CPU.
    Defines directory paths for lunar and Martian data and images.

Utility Functions
1. convert_rel_to_abs_time

```python

def convert_rel_to_abs_time(start_time, time_rel):
    return (start_time + timedelta(seconds=float(time_rel))).strftime('%Y-%m-%dT%H:%M:%S.%f')
```
Explanation:

    Purpose: Converts a relative time (in seconds) to an absolute timestamp by adding it to a start time.
    Parameters:
        start_time: The initial datetime object.
        time_rel: Relative time in seconds.
    Returns: A string representing the absolute time in a specific format.

Optimization Suggestion:

    Ensure that start_time is a datetime object. If start_time might be a string, parse it to datetime first.

2. apply_bandpass_filter

```python

def apply_bandpass_filter(trace, sampling_rate, freqmin=0.5, freqmax=3.0):
    sos = signal.butter(4, [freqmin, freqmax], btype='bandpass', fs=sampling_rate, output='sos')
    return signal.sosfilt(sos, trace)
```
Explanation:

    Purpose: Applies a Butterworth bandpass filter to a signal trace.
    Parameters:
        trace: The signal data as a 1D array.
        sampling_rate: The sampling frequency of the signal.
        freqmin: Minimum frequency for the bandpass filter.
        freqmax: Maximum frequency for the bandpass filter.
    Returns: Filtered signal.

Optimization Suggestion:

    Precompute the filter coefficients if the function is called multiple times with the same parameters to avoid redundant calculations.

3. load_existing_images

```python

def load_existing_images(image_dir):
    image_files = [os.path.join(root, file) for root, _, files in os.walk(image_dir) for file in files if file.endswith('.png')]
    return image_files
```
Explanation:

    Purpose: Recursively loads all .png image file paths from a directory.
    Parameters:
        image_dir: Directory containing images.
    Returns: A list of image file paths.

Optimization Suggestion:

    Use glob.glob with recursive patterns for potentially faster execution.

Preprocessing Functions
4. preprocess_lunar_data

```python

def preprocess_lunar_data(lunar_data_directory):
    lunar_images = load_existing_images(lunar_data_directory)
    print(f"{len(lunar_images)} lunar images loaded.")
    return lunar_images
```
Explanation:

    Purpose: Loads preprocessed lunar images from a directory.
    Parameters:
        lunar_data_directory: Directory containing lunar images.
    Returns: List of lunar image file paths.

Optimization Suggestion:

    Validate if images are correctly loaded and handle potential exceptions.

5. preprocess_martian_data

```python

def preprocess_martian_data(data_dir, save_dir, combine_images=True):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    martian_images, martian_arrival_times = [], []
    mseed_files = [f for f in os.listdir(data_dir) if f.endswith('.mseed')]

    if len(mseed_files) == 0:
        print("No .mseed files found in the directory.")
        return martian_images, martian_arrival_times

    for filename in mseed_files:
        file_path = os.path.join(data_dir, filename)
        csv_file_path = file_path.replace('.mseed', '.csv')

        if not os.path.exists(csv_file_path):
            print(f"CSV file not found for {filename}: {csv_file_path}")
            continue

        try:
            csv_data = pd.read_csv(csv_file_path)
            arrival_time_rel = csv_data['rel_time(sec)'].iloc[0]
            image_path = plot_and_save_trace_spectrogram(file_path, arrival_time_rel, save_dir, filename, combine_images)
            martian_images.append(image_path)
            martian_arrival_times.append(arrival_time_rel)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    return martian_images, martian_arrival_times
```
Explanation:

    Purpose: Preprocesses Martian seismic data by generating spectrogram images and extracting arrival times.
    Parameters:
        data_dir: Directory containing Martian seismic data (.mseed files).
        save_dir: Directory to save generated images.
        combine_images: Flag to combine images (unused in code).
    Returns:
        martian_images: List of paths to generated images.
        martian_arrival_times: List of relative arrival times.

Optimization Suggestions:

    The function plot_and_save_trace_spectrogram is not defined in the provided code. Ensure this function is implemented.
    Use exception handling to catch specific exceptions.
    Consider multithreading or multiprocessing for processing multiple files concurrently.

CNN Model Definition
6. SpectrogramCNN Class

```python

class SpectrogramCNN(nn.Module):
    def __init__(self):
        super(SpectrogramCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 112 * 112, 128)  # Edit to None if it doesn't work
        self.fc_event = nn.Linear(128, 3)
        self.fc_time = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), 128)
        x = torch.relu(self.fc1(x))
        return self.fc_event(x), self.fc_time(x)
```
Explanation:

    Purpose: Defines a Convolutional Neural Network (CNN) for processing spectrogram images.

    Architecture:
        Convolutional Layers:
            conv1: Convolves input image with 32 filters of size 3x3.
            conv2: Convolves output of conv1 with 64 filters of size 3x3.
        Pooling Layer:
            pool: Applies max pooling with a 2x2 kernel.
        Fully Connected Layers:
            fc1: Fully connected layer with an input size of 64 * 112 * 112 and output size of 128.
            fc_event: Outputs event classification into 3 classes.
            fc_time: Outputs a single value representing time.

    Forward Pass:
        Applies ReLU activation after each convolution.
        Applies pooling after the second convolution.
        Flattens the tensor for the fully connected layer.
        Checks if fc1 is None (which shouldn't be the case here).
        Applies ReLU after fc1.
        Returns outputs from fc_event and fc_time.

Optimization Suggestions:

    Issue with fc1 Initialization:
        The check if self.fc1 is None is redundant because fc1 is already defined.
        However, if the input image size changes, 64 * 112 * 112 may not match the flattened size, causing a size mismatch error.
        Solution: Instead of hardcoding the input size for fc1, compute it dynamically:

        ```python

        def __init__(self):
            super(SpectrogramCNN, self).__init__()
            # ... [same as before]
            self.fc1 = None  # Initialize fc1 as None

        def forward(self, x):
            # ... [same as before]
            x = x.view(x.size(0), -1)
            if self.fc1 is None:
                self.fc1 = nn.Linear(x.size(1), 128)
            # ... [same as before]
```
        This ensures fc1 adapts to the input size dynamically.

    Consider Using Pretrained Models:
        For better performance, consider using a pretrained model like ResNet or VGG with adjusted input channels.

Data Preparation
7. prepare_data_for_training

```python

def prepare_data_for_training(image_files, labels, time_labels, batch_size=32):
    if not image_files:
        return None  # Early exit if no images are provided
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image_tensors = [transform(Image.open(img)) for img in image_files if os.path.exists(img) and img.endswith('.png')]
    if image_tensors:
        X_tensor = torch.stack(image_tensors)
        y_event_tensor = torch.tensor(labels, dtype=torch.long)
        y_time_tensor = torch.tensor(time_labels, dtype=torch.float32)
        dataset = TensorDataset(X_tensor, y_event_tensor, y_time_tensor)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return None
```
Explanation:

    Purpose: Prepares a DataLoader for training by loading images and labels.
    Parameters:
        image_files: List of image file paths.
        labels: List of event labels.
        time_labels: List of time labels.
        batch_size: Batch size for DataLoader.
    Process:
        Applies transformations to images (grayscale, resize, tensor conversion, normalization).
        Stacks image tensors.
        Creates tensors for labels.
        Constructs a TensorDataset and returns a DataLoader.

Optimization Suggestions:

    Handle exceptions when opening images.
    Ensure labels and time_labels are aligned with image_tensors.
    Use torchvision.datasets.ImageFolder if the images are organized in folders by class.

Training Functions
8. train_model

```python

def train_model(model, train_loader, criterion_event, criterion_time, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, event_labels, time_labels in train_loader:
            optimizer.zero_grad()
            event_output, time_output = model(inputs)
            loss_event = criterion_event(event_output, event_labels)
            loss_time = criterion_time(time_output.squeeze(), time_labels)
            loss = loss_event + loss_time
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")
```
Explanation:

    Purpose: Trains the CNN model on the provided training data.
    Parameters:
        model: The CNN model.
        train_loader: DataLoader with training data.
        criterion_event: Loss function for event classification.
        criterion_time: Loss function for time regression.
        optimizer: Optimizer for model parameters.
        num_epochs: Number of training epochs.
    Process:
        Sets the model to training mode.
        Iterates over epochs and batches, computes losses, backpropagates, and updates parameters.
        Prints the average loss per epoch.

Optimization Suggestions:

    Add validation after each epoch to monitor overfitting.
    Implement learning rate scheduling.
    Consider logging with tools like TensorBoard.

9. self_train_on_martian_data

```python

def self_train_on_martian_data(model, martian_data_loader, optimizer, criterion_event, criterion_time, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in martian_data_loader:
            images = batch[0]  # Unpack the images from the batch tuple
            optimizer.zero_grad()

            # Forward pass through the model
            event_output, time_output = model(images)

            # Generate pseudo-labels (predicted event labels)
            _, pseudo_labels = torch.max(event_output, 1)

            # For now, we do not have ground truth for Martian data, so we only optimize on pseudo-labels
            loss_event = criterion_event(event_output, pseudo_labels)
            loss_time = criterion_time(time_output.squeeze(), torch.zeros_like(time_output.squeeze()))  # Use zeros as placeholder

            # Calculate total loss and perform backpropagation
            total_loss = loss_event + loss_time
            total_loss.backward()
            optimizer.step()

            running_loss += total_loss.item()

        print(f"Self-training Epoch {epoch+1}, Loss: {running_loss/len(martian_data_loader)}")
```
Explanation:

    Purpose: Performs self-training on Martian data using pseudo-labels generated by the model.
    Parameters:
        martian_data_loader: DataLoader with Martian images (without labels).
    Process:
        Sets the model to training mode.
        For each batch, predicts labels and uses them as pseudo-labels.
        Computes loss and updates model parameters.
        Prints average loss per epoch.

Optimization Suggestions:

    Be cautious with self-training; incorrect pseudo-labels can degrade model performance.
    Consider using confidence thresholds to select only high-confidence predictions for training.
    Implement techniques like semi-supervised learning methods.

Model Evaluation
10. evaluate_model

```python

def evaluate_model(model, data_loader):
    model.eval()
    event_preds, time_preds, event_true, time_true = [], [], [], []
    with torch.no_grad():
        for images, event_labels, time_labels in data_loader:
            event_output, time_output = model(images)
            _, event_pred_classes = torch.max(event_output, 1)
            event_preds.extend(event_pred_classes.cpu().numpy())
            time_preds.extend(time_output.cpu().numpy())
            event_true.extend(event_labels.cpu().numpy())
            time_true.extend(time_labels.cpu().numpy())
    event_accuracy = accuracy_score(event_true, event_preds)
    time_mae = mean_absolute_error(time_true, time_preds)
    print(f"Validation Event Accuracy: {event_accuracy:.4f}")
    print(f"Validation Time MAE: {time_mae:.4f}")
    ConfusionMatrixDisplay.from_predictions(event_true, event_preds)
    plt.show()
```
Explanation:

    Purpose: Evaluates the model's performance on a validation dataset.
    Parameters:
        data_loader: DataLoader with validation data.
    Process:
        Sets the model to evaluation mode.
        Disables gradient computation.
        Collects predictions and true labels.
        Calculates accuracy and mean absolute error (MAE).
        Displays a confusion matrix.

Optimization Suggestions:

    Use GPU if available to speed up evaluation.
    Consider batch-wise evaluation for large datasets.

Model Save Function
11. save_model_artifacts

```python

def save_model_artifacts(model, model_name='seismic_cnn_model'):
    model_architecture = {
        'conv_layers': [{'in_channels': model.conv1.in_channels, 'out_channels': model.conv1.out_channels, 'kernel_size': model.conv1.kernel_size},
                        {'in_channels': model.conv2.in_channels, 'out_channels': model.conv2.out_channels, 'kernel_size': model.conv2.kernel_size}],
        'fc_layers': [{'in_features': model.fc_event.in_features, 'out_features': model.fc_event.out_features},
                      {'in_features': model.fc_time.in_features, 'out_features': model.fc_time.out_features}]
    }
    with open(f'{model_name}_architecture.json', 'w') as f:
        json.dump(model_architecture, f, indent=4)
    torch.save(model.state_dict(), f'{model_name}_weights.pth')
    torch.save(model, f'{model_name}_full.pth')
    print(f"Model saved to {model_name}_full.pth")
```
Explanation:

    Purpose: Saves the model's architecture and weights for future use.
    Parameters:
        model: The trained CNN model.
        model_name: Base name for saved files.
    Process:
        Extracts model architecture details into a dictionary.
        Saves the architecture as a JSON file.
        Saves the model's state dictionary (weights) and the full model.

Optimization Suggestions:

    For version control, include training hyperparameters in the saved artifacts.
    Consider using torch.save with map_location to specify device.

Additional Utility Functions
12. flatten_image_list

```python

def flatten_image_list(image_list):
    """
    Ensure image list is flat in case there are nested lists of image paths.
    """
    if isinstance(image_list, (list, tuple)) and any(isinstance(i, (list, tuple)) for i in image_list):
        return [item for sublist in image_list for item in sublist]
    return image_list
```
Explanation:

    Purpose: Flattens a nested list of image paths into a single list.
    Parameters:
        image_list: List potentially containing nested lists of image paths.
    Returns: Flattened list of image paths.

Optimization Suggestion:

    Use itertools.chain.from_iterable for potentially faster flattening.

13. prepare_unlabeled_data_loader

```python

def prepare_unlabeled_data_loader(image_files, batch_size=32):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image_tensors = [transform(Image.open(img)) for img in image_files if os.path.exists(img) and img.endswith('.png')]
    if image_tensors:
        X_tensor = torch.stack(image_tensors)
        dataset = TensorDataset(X_tensor)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return None
```
Explanation:

    Purpose: Prepares a DataLoader for unlabeled data (used for self-training).
    Parameters:
        image_files: List of image file paths.
        batch_size: Batch size for DataLoader.
    Process:
        Applies transformations to images.
        Stacks image tensors.
        Creates a TensorDataset without labels.
        Returns a DataLoader.

Optimization Suggestion:

    Handle image loading exceptions.
    Consider creating a custom Dataset class for better flexibility.

Main Function
14. main

```python

def main():
    # Define the paths for preprocessed images
    lunar_data_images_dir = '../../model/model_output/lunar_preprocessed_images/'
    martian_data_images_dir = '../../model/model_output/martian_preprocessed_images/'
    save_dir = martian_data_images_dir  # Directory where Martian images are saved

    # 1. Load pre-generated lunar images
    print(f"Loading pre-generated lunar data from: {lunar_data_images_dir}")
    lunar_images = preprocess_lunar_data(lunar_data_images_dir)

    # Check if any lunar images were loaded
    if not lunar_images:
        print("Error: No lunar images found. Exiting.")
        return  # Exit early if no images found

    lunar_labels = [0] * len(lunar_images)  # Placeholder labels for lunar data
    lunar_times = [0] * len(lunar_images)   # Placeholder arrival times for lunar data

    # Prepare DataLoader for lunar data
    print("Preparing DataLoader for lunar data...")
    train_loader = prepare_data_for_training(lunar_images, lunar_labels, lunar_times)

    if train_loader is None:
        print("Error: No valid data for training. DataLoader creation failed.")
        return  # Exit early if DataLoader creation failed

    # 2. Initialize and train the model on lunar data
    print("Initializing SpectrogramCNN model...")
    model = SpectrogramCNN()
    criterion_event = nn.CrossEntropyLoss()
    criterion_time = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print("Training model on lunar data...")
    train_model(model, train_loader, criterion_event, criterion_time, optimizer)

    # 3. Load pre-generated Martian images
    print(f"Loading pre-generated Martian data from: {martian_data_images_dir}")
    martian_images = load_existing_images(martian_data_images_dir)  # Load images directly

    # Check if any Martian images were loaded
    if not martian_images:
        print("Error: No Martian images found. Exiting.")
        return  # Exit early if no images found

    # Prepare DataLoader for Martian data
    print("Preparing DataLoader for Martian data...")
    martian_data_loader = prepare_unlabeled_data_loader(martian_images)

    if martian_data_loader is None:
        print("Error: No valid data for self-training. DataLoader creation failed.")
        return  # Exit early if DataLoader creation failed

    # 4. Self-train the model on Martian data
    print("Self-training model on Martian data...")
    self_train_on_martian_data(model, martian_data_loader, optimizer, criterion_event, criterion_time)

    # 5. Save the fine-tuned model
    print("Saving the fine-tuned model...")
    save_model_artifacts(model, model_name='../../model/fine_tuned_martian_seismic_cnn')
```
Explanation:

    Purpose: Main execution function orchestrating data loading, model training, self-training, and saving.
    Process:
        Loads lunar and Martian images.
        Prepares DataLoaders.
        Initializes the model and criteria.
        Trains on lunar data.
        Self-trains on Martian data.
        Saves the fine-tuned model.

Optimization Suggestions:

    Add exception handling for potential errors.
    Validate the sizes of data at each step.
    Implement command-line argument parsing for flexibility.

Entry Point

```python

if __name__ == "__main__":
    main()
```
    Purpose: Ensures main() is called when the script is run directly.

CNN Analysis and Hyperparameter Tuning Suggestions
CNN Architecture Analysis

The SpectrogramCNN is designed for processing spectrogram images, which are 2D representations of signal frequencies over time.

Architecture Breakdown:

    Input Layer:
        Accepts grayscale images of size (1, 224, 224) after transformations.

    Convolutional Layers:
        conv1: 1 input channel, 32 output channels, kernel size 3x3, stride 1, padding 1.
        Activation: ReLU.
        conv2: 32 input channels, 64 output channels, kernel size 3x3, stride 1, padding 1.
        Activation: ReLU.

    Pooling Layer:
        Max pooling with kernel size 2x2, reducing the spatial dimensions by half.

    Fully Connected Layers:
        fc1: Input size depends on the output from the convolutional layers (flattened).
        fc_event: Outputs logits for 3 classes (event classification).
        fc_time: Outputs a single value (time regression).

Potential Issues:

    Fixed Input Size for fc1:
        The input size to fc1 is hardcoded as 64 * 112 * 112, assuming input images are resized to (224, 224) and pooled once.
        If the input image size changes, this will cause a dimension mismatch.

Recommendations:

    Dynamic Calculation of Flattened Size:
        Instead of hardcoding, calculate the flattened size based on the input data.

    ```python

    def __init__(self):
        super(SpectrogramCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = None  # Initialize as None
        self.fc_event = None
        self.fc_time = None

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), 128).to(x.device)
            self.fc_event = nn.Linear(128, 3).to(x.device)
            self.fc_time = nn.Linear(128, 1).to(x.device)
        x = torch.relu(self.fc1(x))
        return self.fc_event(x), self.fc_time(x)
```
    Batch Normalization and Dropout:
        Add nn.BatchNorm2d after convolutional layers to normalize activations.
        Add nn.Dropout in the fully connected layers to prevent overfitting.

Hyperparameter Tuning Suggestions

To improve training on black and white spectrograms, consider the following hyperparameter adjustments:

    Learning Rate:
        Start with a smaller learning rate, e.g., 1e-4 instead of 1e-3, to ensure stable convergence.

    Optimizer:
        Experiment with different optimizers like AdamW or SGD with momentum.
        Use weight decay to prevent overfitting.

    Batch Size:
        Adjust batch size based on dataset size and memory constraints.
        Smaller batch sizes can lead to noisier gradients but may generalize better.

    Number of Epochs:
        Increase the number of epochs to allow the model to learn more complex patterns.
        Use early stopping based on validation loss to prevent overfitting.

    Data Augmentation:
        Apply transformations like random rotations, flips, or noise addition to augment the dataset.
        This can help the model generalize better to unseen data.

    ```python

    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    ```
    Normalization:
        Ensure the normalization parameters (mean and std) are appropriate for grayscale images.
        Compute the mean and std from the dataset if possible.

    Model Complexity:
        Increase the depth of the network by adding more convolutional layers.
        Use architectures like ResNet or DenseNet which have shown good performance on image data.

    Loss Functions:
        For time regression, consider using SmoothL1Loss which is less sensitive to outliers than MSELoss.
        For classification, ensure the class imbalance is addressed, possibly by using class weights.

    Regularization:
        Add L2 regularization through the optimizer's weight decay parameter.
        Implement dropout layers to prevent overfitting.

    Learning Rate Scheduling:
        Use learning rate schedulers like ReduceLROnPlateau to reduce the learning rate when the validation loss plateaus.
        Alternatively, use CosineAnnealingLR or StepLR.

    Validation Set:
        Split a portion of the training data into a validation set to monitor performance during training.

    Cross-Validation:
        Use k-fold cross-validation to assess model performance more robustly.

    Model Initialization:
        Use better weight initialization methods, such as Xavier or He initialization.

    Transfer Learning:
        Utilize pretrained models on similar tasks and fine-tune them on your dataset.

    Gradient Clipping:
        Apply gradient clipping to prevent exploding gradients.

Conclusion

The codebase provides a framework for training a CNN on spectrogram images for seismic data analysis. By thoroughly analyzing each function and suggesting optimizations, we've identified areas for improvement in data preprocessing, model architecture, and training procedures.

Focusing on the CNN, dynamic adjustments to the model's layers and parameters will enhance flexibility and performance. Additionally, careful hyperparameter tuning, considering the specific nature of black and white spectrograms, will improve training outcomes.

Implementing the suggested hyperparameter adjustments and optimizations will likely lead to better model performance and generalization on the spectrogram data.